In [2]:
from Arsalan.Preprocessing.preprocessing import return_data
from Arsalan.ReadingFiles.GetFiles import get_files
import tensorflow as tf

2024-04-17 17:57:16.196165: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-17 17:57:16.320508: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 17:57:16.320571: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 17:57:16.321966: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-17 17:57:16.366006: I tensorflow/core/platform/cpu_feature_guar

In [3]:
files = get_files('/home/athar/Files/TUKL_work')
file = files[1]
file

('/home/athar/Files/TUKL_work/TUH EEG Corpus/edf/train/abnormal/01_tcp_ar/aaaaaald_s005_t000.edf',
 1)

In [4]:
data = return_data(file[0]).get_data()
data.shape

(21, 42001)

In [5]:
label = 1
label = tf.one_hot(label, 2)
label

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0., 1.], dtype=float32)>

In [6]:
with tf.io.TFRecordWriter('data.tfrecord') as writer:
    features = {
        'label': tf.train.Feature(float_list=tf.train.FloatList(value=label)),
        'feature': tf.train.Feature(float_list=tf.train.FloatList(value=data))
    }
    example = tf.train.Example(features=tf.train.Features(feature=features))
    writer.write(example.SerializeToString())

In [7]:
def map_fn(serialized_example):
    feature = {
        'label': tf.io.FixedLenFeature([2], tf.float32),
        'feature': tf.io.FixedLenFeature([21, 42001], tf.float32)
    }
    example = tf.io.parse_single_example(serialized_example, feature)
    return example['feature'], example['label']

In [8]:
dataset = tf.data.TFRecordDataset('data.tfrecord')
dataset = dataset.map(map_fn)

In [9]:
dataset

<_MapDataset element_spec=(TensorSpec(shape=(21, 42001), dtype=tf.float32, name=None), TensorSpec(shape=(2,), dtype=tf.float32, name=None))>

In [10]:
dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(21, 42001), dtype=tf.float32, name=None), TensorSpec(shape=(2,), dtype=tf.float32, name=None))>